In [ ]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.7 MB/s eta 0:00:00


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Task-1**

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes

# Function to encrypt using AES
def aes_encrypt(plaintext, key, mode):
    key = key.encode('utf-8')

    if mode == 'ECB':
        cipher = AES.new(key, AES.MODE_ECB)
        ciphertext = cipher.encrypt(pad(plaintext.encode('utf-8'), AES.block_size))
        return ciphertext.hex()

    elif mode == 'CBC':
        iv = get_random_bytes(AES.block_size)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        ciphertext = cipher.encrypt(pad(plaintext.encode('utf-8'), AES.block_size))
        return (iv + ciphertext).hex()

# Function to decrypt using AES
def aes_decrypt(ciphertext, key, mode):
    key = key.encode('utf-8')
    ciphertext = bytes.fromhex(ciphertext)

    if mode == 'ECB':
        cipher = AES.new(key, AES.MODE_ECB)
        plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size).decode('utf-8')
        return plaintext

    elif mode == 'CBC':
        iv = ciphertext[:AES.block_size]
        ciphertext = ciphertext[AES.block_size:]
        cipher = AES.new(key, AES.MODE_CBC, iv)
        plaintext = unpad(cipher.decrypt(ciphertext), AES.block_size).decode('utf-8')
        return plaintext

# Main program
def main():
    print("AES Encryption/Decryption Application")
    mode = input("Select AES mode (ECB/CBC): ").strip().upper()

    if mode not in ['ECB', 'CBC']:
        print("Invalid mode selected. Please select either 'ECB' or 'CBC'.")
        return

    operation = input("Select operation (Encrypt/Decrypt): ").strip().lower()

    if operation not in ['encrypt', 'decrypt']:
        print("Invalid operation. Please select either 'Encrypt' or 'Decrypt'.")
        return

    key = input("Enter the key (16, 24, or 32 bytes): ").strip()
    if len(key) not in [16, 24, 32]:
        print("Invalid key length. Key must be 16, 24, or 32 bytes long.")
        return

    if operation == 'encrypt':
        plaintext = input("Enter the plaintext: ").strip()
        ciphertext = aes_encrypt(plaintext, key, mode)
        print(f"Ciphertext (hex): {ciphertext}")

    elif operation == 'decrypt':
        ciphertext = input("Enter the ciphertext (hex): ").strip()
        try:
            plaintext = aes_decrypt(ciphertext, key, mode)
            print(f"Plaintext: {plaintext}")
        except ValueError:
            print("Decryption failed. Invalid key or ciphertext.")

if __name__ == "__main__":
    main()


AES Encryption/Decryption Application
Select AES mode (ECB/CBC): CBC
Select operation (Encrypt/Decrypt): Encrypt
Enter the key (16, 24, or 32 bytes): 1234567890ABCDEF
Enter the plaintext: My sensitive data that needs encryption.
Ciphertext (hex): d32e24958da17bff6dad5730accf9c5ed200502f1103eb145cef304f2158ee0a5b7b3c9dd6e85abd077e6d5ef40a0ebdf8d3cdc2017f01a38253d1d81ffd3a77


**Task-2**


In [29]:
from Crypto.PublicKey import RSA

#Write key to file
key = RSA.generate(4096)
privateKey = key.exportKey()
file1 = open('/content/drive/MyDrive/490/keyfile.pem', 'wb')
file1.write(privateKey)
file1.close()

#Read key from file
file2 = open('keyfile.pem', 'rb')
key = RSA.importKey(file2.read()) #this is the problem

In [32]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import base64

# Function to generate and write RSA private key to a file
def generate_and_save_key(filename):
    key = RSA.generate(4096)
    private_key = key.export_key()
    with open(filename, 'wb') as key_file:
        key_file.write(private_key)
    print(f"Private key written to {filename}")

# Function to read RSA private key from a file
def read_private_key_from_file(filename):
    with open(filename, 'rb') as key_file:
        return RSA.import_key(key_file.read())

# Function to encrypt using RSA
def rsa_encrypt(plaintext, public_key):
    cipher = PKCS1_OAEP.new(public_key)
    ciphertext = cipher.encrypt(plaintext.encode('utf-8'))
    return base64.b64encode(ciphertext).decode('utf-8')

# Function to decrypt using RSA
def rsa_decrypt(ciphertext, private_key):
    cipher = PKCS1_OAEP.new(private_key)
    ciphertext = base64.b64decode(ciphertext.encode('utf-8'))
    plaintext = cipher.decrypt(ciphertext).decode('utf-8')
    return plaintext

# Main program for encryption/decryption
def main():
    print("RSA Encryption/Decryption Application")

    operation = input("Select operation (Generate Key/Encrypt/Decrypt): ").strip().lower()

    if operation == 'generate key':
        filename = input("Enter the file name to save the key (e.g., keyfile.pem): ").strip()
        generate_and_save_key(filename)

    elif operation == 'encrypt':
        plaintext = input("Enter the plaintext: ").strip()
        public_key_file = input("Enter the RSA key file (PEM format): ").strip()
        try:
            public_key = read_private_key_from_file(public_key_file).publickey()  # Get public key from private key
            ciphertext = rsa_encrypt(plaintext, public_key)
            print(f"Ciphertext (base64): {ciphertext}")
        except ValueError as e:
            print(f"Encryption error: {e}")

    elif operation == 'decrypt':
        ciphertext = input("Enter the ciphertext (base64): ").strip()
        private_key_file = input("Enter the RSA key file (PEM format): ").strip()
        try:
            private_key = read_private_key_from_file(private_key_file)  # Read the private key from file
            plaintext = rsa_decrypt(ciphertext, private_key)
            print(f"Plaintext: {plaintext}")
        except ValueError as e:
            print(f"Decryption error: {e}")

    else:
        print("Invalid operation. Please select 'Generate Key', 'Encrypt', or 'Decrypt'.")

if __name__ == "__main__":
    main()


RSA Encryption/Decryption Application
Select operation (Generate Key/Encrypt/Decrypt): encrypt
Enter the plaintext: Hello I am Groot
Enter the RSA key file (PEM format): /content/drive/MyDrive/490/keyfile.pem
Ciphertext (base64): Oz4m9hZkf0Tg6hEJMXNWAMIsINnCBdOhYlG7vDZQtbf52N8djWoDZ2dHhw/yObOhEaUUWp0+fT+WHEKSqnPsWg6KmU2qayE0OaBjPvwYL4bhfCGFi1Rs5UeCI8mFt8jr78KTwYNG8ursYc/xnkkgpyT1ryINdfEumWwC05scNTezmG+hOsetT2cKTbuCnHjZBxZYkZtx3D66D1HCugH0NUFQQ+Pz7er6jh6Z+DwkPvBxC/xw7HHyma4ppBaN3IXdbL4Mf/eNLMCvBSWenynjcT8GG6HUt4ke1QqcqaT72Vk2PYipLC7vu+HMzUJmIurmQI+FRLuEpbzbOwLYx5whyAGdof3q7U7rfR0t+rqNt5UjJxdkTG8dQ88yxFYdJMt56Nbsl2vbs6zvTZ+ltPmP5dO+Ci3B5isrLM3ULai6Ly707HHQSjBhTF6i9xqNS5j6NYeg6Q10MJis2p86NjO0Xtma/j0e32ns5yZQehGwogAXjuUBXl/ORTawQ148/baMitK3rZmKUmG8wy11UgusSP/A117aQezLaZLlzvAQKbbSabJDv1ysM40H156DCSCOpM8CTPK3Bs8LqC7KtWfKOF2ltZQ34jN3bvUMYNU2n1cQnxptLvFWqJb2LqtQI3zwx/ls3fgKiNlV0es69fua7AVpdwCV7JHzk+3tsQxG6jA=


**Task-3**

In [ ]:
import hashlib

# Function to hash using SHA1
def hash_sha1(plaintext):
    sha1_hash = hashlib.sha1(plaintext.encode('utf-8')).hexdigest()
    return sha1_hash

# Function to hash using SHA256
def hash_sha256(plaintext):
    sha256_hash = hashlib.sha256(plaintext.encode('utf-8')).hexdigest()
    return sha256_hash

# Main program for hashing
def main():
    print("Hashing Application")

    plaintext = input("Enter the plaintext to hash: ").strip()
    hash_mode = input("Select hash mode (SHA1/SHA256): ").strip().upper()

    if hash_mode == 'SHA1':
        print(f"SHA1 Hash: {hash_sha1(plaintext)}")
    elif hash_mode == 'SHA256':
        print(f"SHA256 Hash: {hash_sha256(plaintext)}")
    else:
        print("Invalid hash mode. Please select either 'SHA1' or 'SHA256'.")

if __name__ == "__main__":
    main()


Hashing Application
Enter the plaintext to hash: Hello World
Select hash mode (SHA1/SHA256): SHA1
SHA1 Hash: 0a4d55a8d778e5022fab701977c5d840bbc486d0


**Task-4**

In [34]:
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256
import base64

# Function to read RSA key from a .pem file
def read_key_from_pem_file(filename):
    with open(filename, 'rb') as key_file:
        return RSA.import_key(key_file.read())

# Function to sign a message using RSA private key
def rsa_sign(message, private_key):
    h = SHA256.new(message.encode('utf-8'))
    signature = pkcs1_15.new(private_key).sign(h)
    return base64.b64encode(signature).decode('utf-8')

# Function to verify the signature using RSA public key
def rsa_verify(message, signature, public_key):
    h = SHA256.new(message.encode('utf-8'))
    try:
        pkcs1_15.new(public_key).verify(h, base64.b64decode(signature.encode('utf-8')))
        return True
    except (ValueError, TypeError):
        return False

# Main program for digital signature generation and verification
def main():
    print("RSA Digital Signature Application")

    operation = input("Select operation (Generation/Verification): ").strip().lower()

    if operation == "generation":
        message = input("Enter the message to sign: ").strip()
        private_key_file = input("Enter the path to the RSA private key file (e.g., /path/to/private_key.pem): ").strip()

        try:
            private_key = read_key_from_pem_file(private_key_file)
            signature = rsa_sign(message, private_key)
            print(f"Generated Signature (base64): {signature}")
        except ValueError as e:
            print(f"Signing error: {e}")
        except FileNotFoundError:
            print(f"Error: Private key file '{private_key_file}' not found.")

    elif operation == "verification":
        message = input("Enter the original message: ").strip()
        signature = input("Enter the signature (base64): ").strip()
        public_key_file = input("Enter the path to the RSA public key file (e.g., /path/to/public_key.pem): ").strip()

        try:
            public_key = read_key_from_pem_file(public_key_file)
            if rsa_verify(message, signature, public_key):
                print("Signature is valid and message is authentic.")
            else:
                print("Signature is invalid or the message has been altered.")
        except FileNotFoundError:
            print(f"Error: Public key file '{public_key_file}' not found.")

    else:
        print("Invalid operation. Please select 'Generation' or 'Verification'.")

if __name__ == "__main__":
    main()


RSA Digital Signature Application
Select operation (Generation/Verification): Generation
Enter the message to sign: Hello I am GROOT
Enter the path to the RSA private key file (e.g., /path/to/private_key.pem): /content/drive/MyDrive/490/keyfile.pem
Generated Signature (base64): LylWussRve10Rez0sEiK1EQUlSS0V02WOdKsKIJYfVLeFd4AYEK4yWCTPQUbfpho8hYPm7LcxCq29Yb1JcGx60pYBCTM2+SDG2oCmjbrGTETla8PSWqkfzNfZpth/zB9FaNGDBp5VKZ6/KqrrLV6d0Rk86tXa0p53SRFCUb0H9lIAGk+Uy79IA/76bvVuUeXL1sL9sAVNbo/cQsKMpx9U6+vhzlem24MvlAuLUr/P3qqCY+3SSrpkKwWyL5rjgyz3Hn4vOZyBvEeRuV2zSK8QXrI1x/xbEGqUoNPv7ypjjLyqnOQbtSGZpDu0Jpk6g7WC5YUrSyAV1YIrzQW8WOWwbZDUG6OxiABAq1zL+DHlfMi6iNU/z4u5U6yG1jyV7b2TZa4TuWIi3I6/CUEJqhyD431D04WQOp0uUvDiVTv/shWT0X7MR5OJQ757wH/ZLF7XaN40waGFjbYtmNVMc+fVJZ0w5sJLUxSWUxUR6I6WeJfz117NomXpFLW1UO93aYEIKaMjFdk/wPg/zV13SD8uP/qoXUOKpIDrXory6NaHCH3fnP6uf3+UChGIgKBHl+ZgXimnS6NTOKMJ5o3kx2MarG4Y2v11f/dZS2aPbRYJEExkZl67mHBZkuiLDtiGo2K0aucOKcVnrtpBrqBF+hHdAOOlIHGfsWRmWKRZoIXCjQ=


**Task-5**

In [35]:
from Crypto.Hash import CMAC
from Crypto.Cipher import AES

# Function to generate a MAC using CMAC and AES
def generate_mac(message, secret_key):
    cobj = CMAC.new(secret_key, ciphermod=AES)
    cobj.update(message.encode('utf-8'))
    return cobj.hexdigest()

# Function to verify a MAC using CMAC and AES
def verify_mac(message, mac, secret_key):
    cobj = CMAC.new(secret_key, ciphermod=AES)
    cobj.update(message.encode('utf-8'))
    try:
        cobj.verify(bytes.fromhex(mac))
        print(f"The message '{message}' is authentic.")
    except ValueError:
        print(f"The message '{message}' or the MAC is wrong.")

# Main program for MAC generation and verification
def main():
    print("MAC Generation and Verification Application")

    operation = input("Select operation (Generate/Verify): ").strip().lower()

    secret_key = b'Sixteen byte key'  # Example secret key, 16 bytes for AES

    if operation == "generate":
        message = input("Enter the message to generate MAC for: ").strip()
        mac = generate_mac(message, secret_key)
        print(f"Generated MAC (hex): {mac}")

    elif operation == "verify":
        message = input("Enter the message to verify: ").strip()
        mac = input("Enter the MAC (hex format) to verify: ").strip()
        verify_mac(message, mac, secret_key)

    else:
        print("Invalid operation. Please select 'Generate' or 'Verify'.")

if __name__ == "__main__":
    main()


MAC Generation and Verification Application
Select operation (Generate/Verify): Generate
Enter the message to generate MAC for: Hello I am GROOT
Generated MAC (hex): 5cedbae5bce069d1ae0e37f22141a049
